In [5]:
import pandas as pd
from openpyxl import load_workbook

def copiar_filas_celdas_especificas(archivo_excel, hoja_origen, hoja_destino, mapeo_celdas, filtro=None, num_filas=None, espaciado=8):
    """
    Copia múltiples filas de una hoja a celdas específicas en otra hoja.
    
    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja de origen
    hoja_destino (str): Nombre de la hoja de destino
    mapeo_celdas (dict): Diccionario que mapea columnas de origen a celdas de destino
    filtro (tuple): (columna, valor) para filtrar filas específicas
    num_filas (int): Número de últimas filas a copiar
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        
        # Aplicar filtro si se especifica
        if filtro:
            columna, valor = filtro
            df_filtrado = df_origen[df_origen[columna] == valor]
        else:
            df_filtrado = df_origen
            
        # Seleccionar las últimas n filas si se especifica
        if num_filas:
            filas_a_copiar = df_filtrado.tail(num_filas)
        else:
            filas_a_copiar = df_filtrado
            
        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]
        
        # Procesar cada fila
        for idx, fila in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)  # 18 es el rango original (19-2)
            
            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"
                
                valor = fila[1][columna]
                hoja_dest[nueva_celda] = valor
                
        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"Valores copiados exitosamente para {len(filas_a_copiar)} filas")
        
    except Exception as e:
        print(f"Error al copiar los valores: {str(e)}")





# Define el mapeo entre columnas de origen y celdas de destino
mapeo_celdas = {
    'SEMANA': 'B2',      # El valor de la columna 'Nombre' irá a la celda B2
    'PRESIDENCIA': 'O2',        # El valor de la columna 'Edad' irá a la celda D7
    'ORACIÓN': 'O3',
    'TESOROS DE LA BIBLIA': 'O5',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',
    'LECTURA DE LA BIBLIA': 'O7',
    'SMM ASIG 1 ESTUD' : 'L9',
    'SMM ASIG 1 ACOMP' : 'O9',
    'SMM ASIG 2 ESTUD' : 'L10',
    'SMM ASIG 2 ACOMP' : 'O10',
    'SMM ASIG 3 ESTUD' : 'L11',
    'SMM ASIG 3 ACOMP' : 'O11',
    'SMM ASIG 4 ESTUD' : 'L12',
    'SMM ASIG 4 ACOMP' : 'O12',
    'NVC PARTE 1' : 'O15',
    'NVC PARTE 2' : 'O16',
    'ESUDIO LIBRO' : 'O17',
    'LECTOR LIBRO': 'O18',
    'ORACIÓN FINAL': 'O19'
}

# Ejemplo de uso para filtrar por mes

copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    filtro=("MES", "MARZO")
)


# Ejemplo de uso para últimas 4 filas
"""
copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    num_filas=4
)
"""

Valores copiados exitosamente para 5 filas


'\ncopiar_filas_celdas_especificas(\n    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",\n    hoja_origen="BD ASIG",\n    hoja_destino="Formato",\n    mapeo_celdas=mapeo_celdas,\n    num_filas=4\n)\n'

# Extracción de Datos de Web wol.jw.org - Semanas Reunión

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

def extraer_informacion(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer la semana (el primer h1 con id="p1")
    semana_element = soup.find('h1', id='p1')
    semana = semana_element.get_text(strip=True) if semana_element else "Semana no encontrada"
    
    # Extraer el libro de estudio (ahora correctamente como h2 con id="p2")
    libro_element = soup.find('h2', id='p2')
    libro = "Libro no encontrado"
    if libro_element:
        # Intentar encontrar el texto dentro de un strong o directamente
        strong_tag = libro_element.find('strong')
        if strong_tag:
            libro = strong_tag.get_text(strip=True)
        else:
            libro = libro_element.get_text(strip=True)
    
    # Extraer la canción inicial (en el elemento h3 con id="p3")
    cancion_element = soup.find('h3', id='p3')
    cancion = "Canción no encontrada"
    if cancion_element:
        cancion_text = cancion_element.get_text(strip=True)
        # Extraer el número de la canción usando expresión regular
        match = re.search(r'Canción\s+(\d+)', cancion_text)
        if match:
            cancion = f"Canción {match.group(1)}"
    
    # Extraer el tema de Tesoros de la Biblia (el elemento con id="p5")
    tesoros_element = soup.find(['h5', 'h4', 'h3'], id='p5')
    tesoros = tesoros_element.get_text(strip=True) if tesoros_element else "Tesoros no encontrados"
    
    # Buscar la segunda canción (generalmente en la sección SEAMOS MEJORES MAESTROS)
    todas_h3 = soup.find_all('h3')
    segunda_cancion = "Segunda canción no encontrada"
    for h3 in todas_h3:
        texto = h3.get_text(strip=True)
        if "Canción" in texto and texto != cancion_text:
            match = re.search(r'Canción\s+(\d+)', texto)
            if match:
                segunda_cancion = f"Canción {match.group(1)}"
                break
    
    # Buscar la tercera canción (generalmente al final)
    tercera_cancion = "Tercera canción no encontrada"
    for h3 in reversed(todas_h3):
        texto = h3.get_text(strip=True)
        if "Canción" in texto and texto != cancion_text and segunda_cancion not in texto:
            match = re.search(r'Canción\s+(\d+)', texto)
            if match:
                tercera_cancion = f"Canción {match.group(1)}"
                break
    
    return {
        "semana": semana,
        "libro": libro,
        "cancion_inicial": cancion,
        "tesoros": tesoros,
        "segunda_cancion": segunda_cancion,
        "tercera_cancion": tercera_cancion
    }

# Ruta al archivo Excel existente
archivo_excel = "Programación VMC_Septiembre-2024-2025.xlsx"
nombre_hoja = "Sheet1"

# Lista de URLs
urls = [
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/10",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/11",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/12",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/13",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/14"
]

# Lista para almacenar los datos
datos = []

# Extraer información de cada URL
for url in urls:
    try:
        print(f"Procesando: {url}")
        info = extraer_informacion(url)
        
        # Información básica para el DataFrame
        datos_basicos = {
            "Semana": info["semana"],
            "Libro": info["libro"],
            "Canción Inicial": info["cancion_inicial"],
            "Tesoros de la Biblia": info["tesoros"],
            "Segunda Canción": info["segunda_cancion"],
            "Canción Final": info["tercera_cancion"]
        }
        datos.append(datos_basicos)
        
        # Mostrar información detallada
        print("\n--- Información Extraída ---")
        for clave, valor in info.items():
            print(f"{clave.capitalize()}: {valor}")
        print("--------------------------\n")
        
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
    
    time.sleep(1)  # Pausa entre solicitudes

# Crear un DataFrame con los datos
df = pd.DataFrame(datos)

# Verificar si el archivo Excel existe
if os.path.exists(archivo_excel):
    try:
        # Leer todas las hojas existentes
        with pd.ExcelWriter(archivo_excel, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=nombre_hoja, index=False)
        print(f"\nDatos agregados exitosamente a la hoja '{nombre_hoja}' en el archivo '{archivo_excel}'")
        
    except Exception as e:
        print(f"Error al agregar datos al archivo Excel existente: {e}")
        
        # Guardar como respaldo si falla
        nuevo_archivo = "Info_Reuniones_Backup.xlsx"
        df.to_excel(nuevo_archivo, index=False)
        print(f"Datos guardados en archivo de respaldo: '{nuevo_archivo}'")
else:
    print(f"¡Atención! El archivo '{archivo_excel}' no existe en la ubicación actual.")
    
    # Guardar en un nuevo archivo si no existe
    df.to_excel("Info_Reuniones_Nuevas.xlsx", index=False)
    print(f"Datos guardados en un nuevo archivo: 'Info_Reuniones_Nuevas.xlsx'")


# Mostrar la tabla
print("\nResumen de información extraída:")
print(df)

Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/10

--- Información Extraída ---
Semana: 3-9 DE MARZO
Libro: PROVERBIOS 3
Cancion_inicial: Canción 8
Tesoros: 1. Demuestre que confía en Jehová
Segunda_cancion: Canción 124
Tercera_cancion: Canción 57
--------------------------

Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/11

--- Información Extraída ---
Semana: 10-16 DE MARZO
Libro: PROVERBIOS 4
Cancion_inicial: Canción 36
Tesoros: 1. “Protege tu corazón”
Segunda_cancion: Canción 16
Tercera_cancion: Canción 76
--------------------------

Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/12

--- Información Extraída ---
Semana: 17-23 DE MARZO
Libro: PROVERBIOS 5
Cancion_inicial: Canción 122
Tesoros: 1. Manténgase lejos de la inmoralidad sexual
Segunda_cancion: Canción 121
Tercera_cancion: Canción 3
--------------------------

Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/13

--- Información Extraída ---
Semana: 24-30 DE MARZO
Libro: PROV

# Copiar informacón de la reunión

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.cell.cell import MergedCell

def copiar_info_reunion(archivo_excel, hoja_origen="Info-reunión", hoja_destino="Formato", espaciado=8):
    """
    Copia la información de reuniones de la hoja Info-reunión a celdas específicas en la hoja Formato,
    manejando correctamente las celdas combinadas.
    
    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja que contiene la información de reuniones
    hoja_destino (str): Nombre de la hoja donde se copiará la información
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        
        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]
        
        # Definir el mapeo de columnas a celdas
        mapeo_celdas = {
            'Semana': 'B2',           # Semana
            'Libro': 'B4',            # Libro de estudio
            'Canción Inicial': 'O4',  # Primera canción
            'Tesoros de la Biblia': 'O5',  # Tesoros
            'Segunda Canción': 'O13',      # Segunda canción
            'Canción Final': 'O20'         # Tercera canción
        }
        
        def encontrar_celda_principal(hoja, celda_combinada):
            """
            Encuentra la celda principal de un rango de celdas combinadas.
            """
            for rango in hoja.merged_cells.ranges:
                if celda_combinada.coordinate in rango:
                    return hoja.cell(row=rango.min_row, column=rango.min_col)
            return celda_combinada

        # Procesar cada fila
        for idx, fila in enumerate(df_origen.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            
            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"
                
                celda = hoja_dest[nueva_celda]
                valor = fila[1][columna]

                # Si es una celda combinada, encontrar la celda principal
                if isinstance(celda, MergedCell):
                    celda_principal = encontrar_celda_principal(hoja_dest, celda)
                    if celda_principal:
                        celda_principal.value = valor
                else:
                    celda.value = valor
                
        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"Información copiada exitosamente para {len(df_origen)} semanas")
        
    except Exception as e:
        print(f"Error al copiar la información: {str(e)}")
    finally:
        wb.close()

# Ejemplo de uso
if __name__ == "__main__":
    archivo = "Programación VMC_Septiembre-2024-2025.xlsx"
    copiar_info_reunion(archivo)

Error al copiar la información: 'MergedCell' object attribute 'value' is read-only
